In [1]:
import pickle as pkl

with open('rawid2hero.pkl', 'rb') as f:
    rawid2hero = pkl.load(f)

In [2]:
name2id = {}
for idx, h in enumerate(rawid2hero.values()):
    name = h['name'][14:].replace('_', '-')
    name2id[name] = idx

In [3]:
name2id

{'antimage': 0,
 'axe': 1,
 'bane': 2,
 'bloodseeker': 3,
 'crystal-maiden': 4,
 'drow-ranger': 5,
 'earthshaker': 6,
 'juggernaut': 7,
 'mirana': 8,
 'morphling': 9,
 'nevermore': 10,
 'phantom-lancer': 11,
 'puck': 12,
 'pudge': 13,
 'razor': 14,
 'sand-king': 15,
 'storm-spirit': 16,
 'sven': 17,
 'tiny': 18,
 'vengefulspirit': 19,
 'windrunner': 20,
 'zuus': 21,
 'kunkka': 22,
 'lina': 23,
 'lion': 24,
 'shadow-shaman': 25,
 'slardar': 26,
 'tidehunter': 27,
 'witch-doctor': 28,
 'lich': 29,
 'riki': 30,
 'enigma': 31,
 'tinker': 32,
 'sniper': 33,
 'necrolyte': 34,
 'warlock': 35,
 'beastmaster': 36,
 'queenofpain': 37,
 'venomancer': 38,
 'faceless-void': 39,
 'skeleton-king': 40,
 'death-prophet': 41,
 'phantom-assassin': 42,
 'pugna': 43,
 'templar-assassin': 44,
 'viper': 45,
 'luna': 46,
 'dragon-knight': 47,
 'dazzle': 48,
 'rattletrap': 49,
 'leshrac': 50,
 'furion': 51,
 'life-stealer': 52,
 'dark-seer': 53,
 'clinkz': 54,
 'omniknight': 55,
 'enchantress': 56,
 'huskar': 

In [4]:
from py2neo import Graph, Node, Relationship

graph = Graph("bolt://localhost:7687", auth=("neo4j", "1234"))

In [5]:
r_types = list(graph.schema.relationship_types)
r_types

['BestAgainst',
 'AtLane',
 'PrimaryAttributeIs',
 'WorstAgainst',
 'HasLabel',
 'UsesItem']

In [6]:
r_emb_dim = [5, 3, 2, 5, 5, 5]

In [9]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

def to_matrix(pairs):
    le = LabelEncoder()
    pairs[:, 1] = le.fit_transform(pairs[:, 1])
    m = np.zeros((len(name2id), len(le.classes_)))
    for p in pairs:
        h_str, e = p
        h = name2id[h_str]
        m[h][e] = 1
    return m

def graph_laplacian(y):
    d_v = np.sum(y, axis=0)
    d_norm = 1 / np.sqrt(d_v)
    d = np.diag(d_norm)
    return d @ y @ d

def eig_decomp(m, F):
    e_vals, e_vecs = np.linalg.eig(m)

    pairs = [(e_vals[i], e_vecs[:, i]) for i in range(m.shape[0])]
    pairs = sorted(pairs, key=lambda x:x[0], reverse=True)

    p = np.stack([pairs[i][1] for i in range(F)]).T
    p = np.real(p)
    
    return p

In [21]:
features = []
for t, f in zip(r_types, r_emb_dim):
    df = graph.run("MATCH x=(n:Hero)-[r]->(e) RETURN n.name, e.name").to_data_frame()
    m = to_matrix(np.array(df))
    l = graph_laplacian(m @ m.T)
    p = eig_decomp(l, f)
    features.append(p)
kg_embed = np.concatenate(features, 1)

In [23]:
kg_embed.shape

(123, 25)

In [25]:
np.save('kg_embed', kg_embed)

In [19]:
sim = np.zeros_like(l)
for i in range(p.shape[0]):
    for j in range(p.shape[0]):
        sim[int(i)][int(j)] = np.dot(p[i], p[j]) / (np.linalg.norm(p[i]) * np.linalg.norm(p[j]))

In [20]:
np.argsort(sim[7])

array([ 34,  32,  63,  21,  74,  12,  41,  50, 103, 117,  20,  72,  99,
        15,  23,  24,  31,  73,  16, 119,   6, 115,  36,  84,  47,  37,
        96, 110,  43, 120,  66,  77,  88,  98,  27,  94,  76,  62, 102,
        53, 113,  13,  48,  22,   1,  90,  49,  35, 122,  10,  71,  18,
        85,  82,  28,  95,  25,   4,  67,  81,   2,  29,  26, 111,  55,
        97, 118,  38,  83,  58,  86,  64, 106, 109,   8,  51, 104,  45,
       105,  17,  57,  89, 100,  40,  56, 108, 121,  44, 101,  69,  80,
        78,  14,  60,  19, 114,  79,  33,  70,  87, 116,  92,  52,  11,
        59,  75,   9,  61,  68,  54,  46,  93, 112,   5, 107,   3,  91,
        42,  65,  30,   0,  39,   7])